##### Create a Vertex Search App
[Follow These Steps](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#unstructured-data_1) to leverage Vertex Agent Builder to create a Search App with unstructured data store on Google Cloud Console. 

Once the Search App is created, you can leverage the [Vertex AI Agent Builder Python API reference documentation](https://cloud.google.com/python/docs/reference/discoveryengine/latest) to get the search results as shown below. 

In [ ]:
!pip install google-cloud-discoveryengine

In [ ]:
from typing import List
from google.api_core.client_options import ClientOptions

In [ ]:
from google.cloud import discoveryengine_v1 as discoveryengine

In [ ]:
project_id = "YOUR_PROJECT_ID"
location = "global"          # Values: "global", "us", "eu"
engine_id = "YOUR_VERTEX_SEARCH_APP_ID"
search_query = "When does Alphabet plan to get to net zero?"

In [ ]:
CUSTOM_PROMPT = """
    <PERSONA_AND_GOAL>
      You are a helpful assistant knowledgeable about Alphabet's quarterly earning reports.
      Be mindful of the timeframe user inputs
      Help user with their queries related with Alphabet
      Only respond with relevant information available in Grounding Knowledge snippet.
      If no relevant snippet is available, respond with you don't know
      Do not make up information
    </PERSONA_AND_GOAL>
"""

In [ ]:
def search_spec():
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True),
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=10,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
            model_prompt_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelPromptSpec(
                preamble=CUSTOM_PROMPT),
            model_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
                version="stable",),
        ),
    )
    return content_search_spec

In [ ]:
def search_sample( project_id: str, location: str, engine_id: str, search_query: str):

    client_options = (ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    client = discoveryengine.SearchServiceClient(client_options=client_options)

    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_config"

    content_search_spec = search_spec()

    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=10,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )
    response = client.search(request)

    return response

In [ ]:
search_response = search_sample(project_id, location, engine_id, search_query)

In [ ]:
search_response.summary.summary_with_metadata

summary: "Alphabet plans to achieve net-zero emissions across all of its operations and value chain by 2030."
citation_metadata {
  citations {
    end_index: 98
    sources {
    }
    sources {
      reference_index: 1
    }
    sources {
      reference_index: 2
    }
    sources {
      reference_index: 3
    }
  }
}
references {
  title: " GOOG 10-K 2023"
  document: "projects/801452371447/locations/global/collections/default_collection/dataStores/alphabet-contracts1-id/branches/0/documents/8fe77e14325ccbf8ea9dbb1043ae96d7"
}
references {
  title: "2022_alphabet_annual_report"
  document: "projects/801452371447/locations/global/collections/default_collection/dataStores/alphabet-contracts1-id/branches/0/documents/276cee4c4086600303bc561483691f9a"
}
references {
  title: "GOOG 10-K Q4 2022"
  document: "projects/801452371447/locations/global/collections/default_collection/dataStores/alphabet-contracts1-id/branches/0/documents/e528afa38b3a739bc98bfff0e840fe98"
}
references {
  title: